In [7]:
import ee
import numpy as np
import pandas as pd

from earthshot import mon_stats
from earthshot import water_viz as vis
from earthshot import normalize as norm
from earthshot import water_common as common
from statistics import mean

import geemap.eefolium as geemap
import folium
from folium import plugins
import geetools
from geetools import batch

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ee.Initialize()

In [49]:
#Join beaver dam tables

CA_Dam_Data_Present = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/06_CaliforniaDams_Boxes_yes_BeaverDam')
CA_Dam_Data_Absent = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/06_CaliforniaDams_Boxes_no_BeaverDam')
CO_Dam_Data_Present = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/05_ColoradoDams_Boxes_yes_BeaverDam')
CO_Dam_Data_Absent = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/05_ColoradoDams_Boxes_no_BeaverDam')
CO_Dam_Data_Intersects = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/05_ColoradoDams_Boxes_allIntersect')
ID_Dam_Data_Present = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/04_IdahoDams_Boxes_yes_BeaverDam')
ID_Dam_Data_Absent = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/04_IdahoDams_Boxes_no_BeaverDam')
ID_Dam_data_Intersects = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/04_IdahoDams_Boxes_allIntersect')
WY_Dam_Data_Present = (ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/01_WyomingDams_Boxes_yes_BeaverDam')
                       .merge(ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/02_WyomingDams_Boxes_yes_BeaverDam')))
WY_Dam_Data_Absent = (ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/01_WyomingDams_Boxes_no_BeaverDam')
                      .merge(ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/02_WyomingDams_Boxes_no_BeaverDam')))
WY_Dam_Data_Intersects = (ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/01_WyomingDams_Boxes_allIntersect')
                        .merge(ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/02_WyomingDams_Boxes_allIntersect')))
OR_Dam_Data_Present = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/03_OregonDams_Boxes_yes_BeaverDam')
OR_Dam_Data_Intersects = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/03_OregonDams_Boxes_allIntersect')
OR_Dam_Data_Absent = ee.FeatureCollection('users/amgadellaboudy/Beaver_Dams_A/03_OregonDams_Boxes_no_BeaverDam')

dam_data_present = (CA_Dam_Data_Present.merge(CO_Dam_Data_Present).merge(ID_Dam_Data_Present).merge(WY_Dam_Data_Present).merge(OR_Dam_Data_Present)).map(set_present)

dam_data_absent = CA_Dam_Data_Absent.merge(CO_Dam_Data_Absent).merge(ID_Dam_Data_Absent).merge(WY_Dam_Data_Absent).merge(OR_Dam_Data_Absent).map(set_absent)

dam_data_intersects = CO_Dam_Data_Intersects.merge(ID_Dam_data_Intersects).merge(WY_Dam_Data_Intersects).merge(OR_Dam_Data_Intersects)

In [59]:
dataset = (ee.ImageCollection('USDA/NAIP/DOQQ')
            .filter(ee.Filter.date('2016-01-01', '2018-12-31')).mean())

#split = 0.99
#random = dam_data.randomColumn()
#dam_data_split = random.filter(ee.Filter.gt('random', split))

In [50]:
coll = dataset.clip(dam_data_present)

AttributeError: 'Image' object has no attribute 'size'

In [60]:
collection_intersects = ee.ImageCollection(dam_data_intersects.map(clip))
print(collection_intersects.size().getInfo())

collection_absent = ee.ImageCollection(dam_data_absent.map(clip))
print(collection_absent.size().getInfo())

1238
18243


In [61]:
#constant1 = dataset.clipToCollection(dam_data)
bbox_conus = ee.Geometry.BBox(-125, 30, -103, 49)

#collection = ee.ImageCollection([constant1])

def clip(feature):
    return dataset.clip(feature)

collection_merged = ee.ImageCollection(dam_data_present.map(clip))


#batch.imagecollection.toAsset(collection_merged, assetPath = 'users/amgadellaboudy/beaver_dam_coll', scale = 1)

size = collection_merged.size().getInfo()
clist = collection_merged.toList(size)

for i in range(size):
    image = ee.Image(clist.get(i))
    iid = image.id().getInfo()
    name = iid
    print('downloading '+name)
    xx = ee.batch.Export.image.toDrive(image, description = 'Beaver_Dam_Export', folder = 'Beaver Dam Images- Present', fileNamePrefix = name, scale = 0.50).start()



downloading 1_1_1_1_00000000000000000010
downloading 1_1_1_1_00000000000000000049
downloading 1_1_1_1_00000000000000000066
downloading 1_1_1_1_00000000000000000061
downloading 1_1_1_1_00000000000000000052
downloading 1_1_1_1_0000000000000000003d
downloading 1_1_1_1_00000000000000000036
downloading 1_1_1_1_00000000000000000056
downloading 1_1_1_1_00000000000000000011
downloading 1_1_1_1_0000000000000000004d
downloading 1_1_1_1_0000000000000000000c
downloading 1_1_1_1_0000000000000000000e
downloading 1_1_1_1_0000000000000000000f
downloading 1_1_1_1_00000000000000000000
downloading 1_1_1_1_0000000000000000000a
downloading 1_1_1_1_0000000000000000000d
downloading 1_1_1_1_00000000000000000007
downloading 1_1_1_1_00000000000000000008
downloading 1_1_1_1_00000000000000000009
downloading 1_1_1_1_00000000000000000006
downloading 1_1_1_1_0000000000000000004b
downloading 1_1_1_1_0000000000000000004c
downloading 1_1_1_1_00000000000000000038
downloading 1_1_1_1_00000000000000000039
downloading 1_1_

downloading 1_1_1_2_00000000000000000028
downloading 1_1_1_2_00000000000000000029
downloading 1_1_1_2_0000000000000000002b
downloading 1_1_1_2_0000000000000000002d
downloading 1_1_1_2_00000000000000000031
downloading 1_1_1_2_00000000000000000033
downloading 1_1_1_2_00000000000000000037
downloading 1_1_1_2_00000000000000000038
downloading 1_1_1_2_00000000000000000039
downloading 1_1_1_2_0000000000000000003a
downloading 1_1_1_2_0000000000000000003c
downloading 1_1_1_2_0000000000000000003d
downloading 1_1_1_2_0000000000000000004f
downloading 1_1_1_2_00000000000000000050
downloading 1_1_1_2_00000000000000000051
downloading 1_1_1_2_00000000000000000052
downloading 1_1_1_2_00000000000000000053
downloading 1_1_1_2_00000000000000000054
downloading 1_1_1_2_00000000000000000055
downloading 1_1_1_2_00000000000000000056
downloading 1_1_1_2_00000000000000000058
downloading 1_1_1_2_00000000000000000059
downloading 1_1_1_2_0000000000000000005b
downloading 1_1_1_2_0000000000000000005c
downloading 1_1_

downloading 1_2_1_0000000000000000010a
downloading 1_2_1_0000000000000000010b
downloading 1_2_1_0000000000000000010c
downloading 1_2_1_0000000000000000010d
downloading 1_2_1_0000000000000000010e
downloading 1_2_1_0000000000000000010f
downloading 1_2_1_00000000000000000110
downloading 1_2_1_000000000000000000eb
downloading 1_2_1_000000000000000000ec
downloading 1_2_1_000000000000000000ed
downloading 1_2_1_000000000000000000e3
downloading 1_2_1_000000000000000000e4
downloading 1_2_1_000000000000000000e5
downloading 1_2_1_000000000000000000e6
downloading 1_2_1_000000000000000000e8
downloading 1_2_1_000000000000000000e9
downloading 1_2_1_000000000000000000ea
downloading 1_2_1_00000000000000000118
downloading 1_2_1_0000000000000000011c
downloading 1_2_1_0000000000000000011e
downloading 1_2_1_0000000000000000011f
downloading 1_2_1_00000000000000000122
downloading 1_2_1_00000000000000000123
downloading 1_2_1_00000000000000000124
downloading 1_2_1_00000000000000000167
downloading 1_2_1_0000000

downloading 1_2_1_0000000000000000003a
downloading 1_2_1_0000000000000000003b
downloading 1_2_1_0000000000000000003c
downloading 1_2_1_0000000000000000003d
downloading 1_2_1_0000000000000000003e
downloading 1_2_1_0000000000000000003f
downloading 1_2_1_00000000000000000040
downloading 1_2_1_00000000000000000041
downloading 1_2_1_00000000000000000042
downloading 1_2_1_00000000000000000043
downloading 1_2_1_00000000000000000044
downloading 1_2_1_00000000000000000045
downloading 1_2_1_00000000000000000046
downloading 1_2_1_00000000000000000047
downloading 1_2_1_00000000000000000035
downloading 1_2_1_00000000000000000036
downloading 1_2_1_00000000000000000032
downloading 1_2_1_00000000000000000034
downloading 1_2_1_0000000000000000002d
downloading 1_2_1_0000000000000000002e
downloading 1_2_1_0000000000000000002f
downloading 1_2_1_00000000000000000030
downloading 1_2_1_00000000000000000033
downloading 1_2_1_00000000000000000023
downloading 1_2_1_00000000000000000024
downloading 1_2_1_0000000

In [62]:
size = collection_intersects.size().getInfo()
clist = collection_intersects.toList(size)

for i in range(size):
    image = ee.Image(clist.get(i))
    iid = image.id().getInfo()
    name = iid
    print('downloading '+name)
    xx = ee.batch.Export.image.toDrive(image, description = 'Beaver_Dam_Export', folder = 'Beaver Dam Images- Intersects', fileNamePrefix = name, scale = 0.50).start()

downloading 1_1_1_00000000000000000012
downloading 1_1_1_00000000000000000011
downloading 1_1_1_00000000000000000000
downloading 1_1_1_000000000000000000ff
downloading 1_1_1_000000000000000000f5
downloading 1_1_1_000000000000000000f6
downloading 1_1_1_000000000000000000f2
downloading 1_1_1_000000000000000000f3
downloading 1_1_1_00000000000000000088
downloading 1_1_1_000000000000000000e7
downloading 1_1_1_000000000000000000e8
downloading 1_1_1_000000000000000000e5
downloading 1_1_1_000000000000000000e6
downloading 1_1_1_00000000000000000001
downloading 1_1_1_00000000000000000003
downloading 1_1_1_00000000000000000004
downloading 1_1_1_00000000000000000002
downloading 1_1_1_000000000000000000fd
downloading 1_1_1_000000000000000000fe
downloading 1_1_1_00000000000000000100
downloading 1_1_1_000000000000000000e9
downloading 1_1_1_000000000000000000ea
downloading 1_1_1_0000000000000000008e
downloading 1_1_1_0000000000000000008f
downloading 1_1_1_00000000000000000048
downloading 1_1_1_0000000

downloading 1_1_1_00000000000000000051
downloading 1_1_1_00000000000000000052
downloading 1_1_1_00000000000000000044
downloading 1_1_1_00000000000000000045
downloading 1_1_1_00000000000000000046
downloading 1_1_1_00000000000000000047
downloading 1_1_1_00000000000000000016
downloading 1_1_1_00000000000000000019
downloading 1_1_1_00000000000000000013
downloading 1_1_1_00000000000000000014
downloading 1_1_1_00000000000000000015
downloading 1_1_1_00000000000000000010
downloading 1_1_1_0000000000000000000a
downloading 1_1_1_0000000000000000000f
downloading 1_1_1_0000000000000000000c
downloading 1_1_1_0000000000000000000d
downloading 1_1_1_000000000000000000d1
downloading 1_1_1_000000000000000000d2
downloading 1_1_1_000000000000000000ca
downloading 1_1_1_000000000000000000cb
downloading 1_1_1_000000000000000000cc
downloading 1_1_1_000000000000000000cd
downloading 1_1_1_000000000000000000ce
downloading 1_1_1_000000000000000000cf
downloading 1_1_1_000000000000000000d5
downloading 1_1_1_0000000

downloading 1_2_1_000000000000000001d8
downloading 1_2_1_000000000000000001d9
downloading 1_2_1_000000000000000001da
downloading 1_2_1_000000000000000001db
downloading 1_2_1_000000000000000001e4
downloading 1_2_1_000000000000000001e5
downloading 1_2_1_000000000000000001e6
downloading 1_2_1_000000000000000001e7
downloading 1_2_1_000000000000000001e0
downloading 1_2_1_000000000000000001e2
downloading 1_2_1_000000000000000001e3
downloading 1_2_1_000000000000000001dc
downloading 1_2_1_000000000000000001dd
downloading 1_2_1_000000000000000001de
downloading 1_2_1_000000000000000001df
downloading 1_2_1_000000000000000001e1
downloading 1_2_1_000000000000000001ac
downloading 1_2_1_000000000000000001ad
downloading 1_2_1_000000000000000001ae
downloading 1_2_1_000000000000000000ea
downloading 1_2_1_000000000000000000ff
downloading 1_2_1_00000000000000000104
downloading 1_2_1_00000000000000000108
downloading 1_2_1_00000000000000000114
downloading 1_2_1_00000000000000000123
downloading 1_2_1_0000000

downloading 1_2_1_00000000000000000239
downloading 1_2_1_0000000000000000023a
downloading 1_2_1_0000000000000000023b
downloading 1_2_1_0000000000000000023d
downloading 1_2_1_00000000000000000245
downloading 1_2_1_00000000000000000248
downloading 1_2_1_00000000000000000249
downloading 1_2_1_0000000000000000023c
downloading 1_2_1_0000000000000000023f
downloading 1_2_1_00000000000000000242
downloading 1_2_1_00000000000000000243
downloading 1_2_1_0000000000000000024a
downloading 1_2_1_0000000000000000020c
downloading 1_2_1_0000000000000000020d
downloading 1_2_1_0000000000000000020e
downloading 1_2_1_00000000000000000213
downloading 1_2_1_00000000000000000214
downloading 1_2_1_00000000000000000216
downloading 1_2_1_00000000000000000218
downloading 1_2_1_0000000000000000019a
downloading 1_2_1_00000000000000000219
downloading 1_2_1_0000000000000000021a
downloading 1_2_1_00000000000000000240
downloading 1_2_1_00000000000000000241
downloading 1_2_1_00000000000000000244
downloading 1_2_1_0000000

downloading 1_2_1_000000000000000000e8
downloading 1_2_1_000000000000000000e9
downloading 1_2_1_000000000000000000d4
downloading 1_2_1_000000000000000000e1
downloading 1_2_1_000000000000000000e2
downloading 1_2_1_000000000000000000c4
downloading 1_2_1_000000000000000000c5
downloading 1_2_1_000000000000000000c6
downloading 1_2_1_000000000000000000c7
downloading 1_2_1_000000000000000000c8
downloading 1_2_1_000000000000000000cc
downloading 1_2_1_000000000000000000cd
downloading 1_2_1_000000000000000000ce
downloading 1_2_1_000000000000000000cf
downloading 1_2_1_000000000000000000d0
downloading 1_2_1_000000000000000000d1
downloading 1_2_1_000000000000000000d2
downloading 1_2_1_000000000000000000d5
downloading 1_2_1_000000000000000000d6
downloading 1_2_1_000000000000000000b0
downloading 1_2_1_000000000000000000b1
downloading 1_2_1_000000000000000000b2
downloading 1_2_1_000000000000000000b3
downloading 1_2_1_000000000000000000b4
downloading 1_2_1_000000000000000000b5
downloading 1_2_1_0000000

downloading 1_2_2_00000000000000000069
downloading 1_2_2_0000000000000000006a
downloading 1_2_2_0000000000000000006d
downloading 1_2_2_0000000000000000006e
downloading 1_2_2_0000000000000000002c
downloading 1_2_2_0000000000000000002f
downloading 1_2_2_00000000000000000058
downloading 1_2_2_00000000000000000059
downloading 1_2_2_0000000000000000005a
downloading 1_2_2_00000000000000000032
downloading 1_2_2_00000000000000000031
downloading 1_2_2_00000000000000000034
downloading 1_2_2_00000000000000000035
downloading 1_2_2_00000000000000000036
downloading 1_2_2_00000000000000000037
downloading 1_2_2_00000000000000000038
downloading 1_2_2_00000000000000000039
downloading 1_2_2_0000000000000000003a
downloading 1_2_2_00000000000000000028
downloading 1_2_2_0000000000000000002a
downloading 1_2_2_00000000000000000024
downloading 1_2_2_00000000000000000049
downloading 1_2_2_0000000000000000004a
downloading 1_2_2_0000000000000000004b
downloading 1_2_2_0000000000000000004c
downloading 1_2_2_0000000

In [63]:
size = collection_absent.size().getInfo()
clist = collection_absent.toList(size)

for i in range(size):
    image = ee.Image(clist.get(i))
    iid = image.id().getInfo()
    name = iid
    print('downloading '+name)
    xx = ee.batch.Export.image.toDrive(image, description = 'Beaver_Dam_Export', folder = 'Beaver Dam Images- Absent', fileNamePrefix = name, scale = 0.50).start()

downloading 1_1_1_1_000000000000000008fd
downloading 1_1_1_1_000000000000000008fe
downloading 1_1_1_1_000000000000000008ff
downloading 1_1_1_1_000000000000000008f9
downloading 1_1_1_1_000000000000000008fa
downloading 1_1_1_1_000000000000000008fb
downloading 1_1_1_1_000000000000000008f1
downloading 1_1_1_1_000000000000000008f2
downloading 1_1_1_1_000000000000000008f3
downloading 1_1_1_1_000000000000000008f4
downloading 1_1_1_1_000000000000000008f5
downloading 1_1_1_1_000000000000000008f6
downloading 1_1_1_1_000000000000000008f7
downloading 1_1_1_1_000000000000000008f8
downloading 1_1_1_1_000000000000000008fc
downloading 1_1_1_1_0000000000000000090c
downloading 1_1_1_1_0000000000000000090d
downloading 1_1_1_1_0000000000000000090e
downloading 1_1_1_1_00000000000000000904
downloading 1_1_1_1_00000000000000000905
downloading 1_1_1_1_00000000000000000906
downloading 1_1_1_1_00000000000000000907
downloading 1_1_1_1_00000000000000000908
downloading 1_1_1_1_00000000000000000909
downloading 1_1_

downloading 1_1_1_1_000000000000000009f0
downloading 1_1_1_1_000000000000000009f1
downloading 1_1_1_1_000000000000000009f2
downloading 1_1_1_1_000000000000000009f3
downloading 1_1_1_1_00000000000000000a08
downloading 1_1_1_1_00000000000000000a09
downloading 1_1_1_1_00000000000000000a0a
downloading 1_1_1_1_00000000000000000a0c
downloading 1_1_1_1_00000000000000000a0d
downloading 1_1_1_1_00000000000000000a0e
downloading 1_1_1_1_00000000000000000a0f
downloading 1_1_1_1_00000000000000000a04
downloading 1_1_1_1_00000000000000000a05
downloading 1_1_1_1_00000000000000000a0b
downloading 1_1_1_1_000000000000000009d8
downloading 1_1_1_1_000000000000000009dc
downloading 1_1_1_1_000000000000000009dd
downloading 1_1_1_1_000000000000000009de
downloading 1_1_1_1_000000000000000009df
downloading 1_1_1_1_00000000000000000a00
downloading 1_1_1_1_00000000000000000a01
downloading 1_1_1_1_00000000000000000a02
downloading 1_1_1_1_00000000000000000a03
downloading 1_1_1_1_00000000000000000a06
downloading 1_1_

downloading 1_1_1_1_00000000000000000775
downloading 1_1_1_1_0000000000000000077c
downloading 1_1_1_1_0000000000000000089d
downloading 1_1_1_1_0000000000000000089e
downloading 1_1_1_1_000000000000000008c2
downloading 1_1_1_1_000000000000000008c3
downloading 1_1_1_1_000000000000000008c4
downloading 1_1_1_1_00000000000000000891
downloading 1_1_1_1_00000000000000000892
downloading 1_1_1_1_00000000000000000893
downloading 1_1_1_1_00000000000000000894
downloading 1_1_1_1_00000000000000000897
downloading 1_1_1_1_00000000000000000898
downloading 1_1_1_1_000000000000000008ad
downloading 1_1_1_1_000000000000000008ae
downloading 1_1_1_1_000000000000000008af
downloading 1_1_1_1_00000000000000000960
downloading 1_1_1_1_00000000000000000961
downloading 1_1_1_1_00000000000000000962
downloading 1_1_1_1_00000000000000000963
downloading 1_1_1_1_00000000000000000964
downloading 1_1_1_1_00000000000000000965
downloading 1_1_1_1_00000000000000000966
downloading 1_1_1_1_00000000000000000967
downloading 1_1_

downloading 1_1_1_1_000000000000000006d9
downloading 1_1_1_1_000000000000000006da
downloading 1_1_1_1_000000000000000006db
downloading 1_1_1_1_000000000000000006dc
downloading 1_1_1_1_000000000000000006de
downloading 1_1_1_1_000000000000000006df
downloading 1_1_1_1_000000000000000006e0
downloading 1_1_1_1_000000000000000006d3
downloading 1_1_1_1_000000000000000006d4
downloading 1_1_1_1_000000000000000006ed
downloading 1_1_1_1_000000000000000006ee
downloading 1_1_1_1_000000000000000006ef
downloading 1_1_1_1_000000000000000006f0
downloading 1_1_1_1_000000000000000006e9
downloading 1_1_1_1_000000000000000006ea
downloading 1_1_1_1_000000000000000006eb
downloading 1_1_1_1_000000000000000006ec
downloading 1_1_1_1_000000000000000006e1
downloading 1_1_1_1_000000000000000006e5
downloading 1_1_1_1_000000000000000006e6
downloading 1_1_1_1_000000000000000006e7
downloading 1_1_1_1_000000000000000006e8
downloading 1_1_1_1_00000000000000000691
downloading 1_1_1_1_00000000000000000696
downloading 1_1_

downloading 1_1_1_1_00000000000000000487
downloading 1_1_1_1_00000000000000000488
downloading 1_1_1_1_00000000000000000489
downloading 1_1_1_1_0000000000000000048a
downloading 1_1_1_1_0000000000000000048b
downloading 1_1_1_1_0000000000000000048c
downloading 1_1_1_1_0000000000000000048d
downloading 1_1_1_1_0000000000000000048e
downloading 1_1_1_1_0000000000000000048f
downloading 1_1_1_1_00000000000000000490
downloading 1_1_1_1_00000000000000000491
downloading 1_1_1_1_00000000000000000492
downloading 1_1_1_1_00000000000000000493
downloading 1_1_1_1_00000000000000000494
downloading 1_1_1_1_00000000000000000495
downloading 1_1_1_1_00000000000000000496
downloading 1_1_1_1_00000000000000000462
downloading 1_1_1_1_00000000000000000f30
downloading 1_1_1_1_00000000000000000f31
downloading 1_1_1_1_00000000000000000f32
downloading 1_1_1_1_00000000000000000f34
downloading 1_1_1_1_00000000000000000f35
downloading 1_1_1_1_00000000000000000f36
downloading 1_1_1_1_00000000000000000f37
downloading 1_1_

...

downloading 1_1_1_1_00000000000000000bcb
downloading 1_1_1_1_00000000000000000bcf
downloading 1_1_1_1_00000000000000000497
downloading 1_1_1_1_00000000000000000498
downloading 1_1_1_1_00000000000000000499
downloading 1_1_1_1_0000000000000000049a
downloading 1_1_1_1_0000000000000000049b
downloading 1_1_1_1_0000000000000000049c
downloading 1_1_1_1_0000000000000000049d
downloading 1_1_1_1_0000000000000000049e
downloading 1_1_1_1_0000000000000000049f
downloading 1_1_1_1_000000000000000004a0
downloading 1_1_1_1_000000000000000004a1
downloading 1_1_1_1_000000000000000004a2
downloading 1_1_1_1_000000000000000004a3
downloading 1_1_1_1_000000000000000004a4
downloading 1_1_1_1_000000000000000004a5
downloading 1_1_1_1_000000000000000004a6
downloading 1_1_1_1_000000000000000004ca
downloading 1_1_1_1_00000000000000000bcc
downloading 1_1_1_1_000000000000000004a7
downloading 1_1_1_1_000000000000000004ab
downloading 1_1_1_1_000000000000000004ac
downloading 1_1_1_1_000000000000000004ad
downloading 1_1_

downloading 1_1_1_1_00000000000000001106
downloading 1_1_1_1_0000000000000000111f
downloading 1_1_1_1_00000000000000001120
downloading 1_1_1_1_00000000000000001121
downloading 1_1_1_1_00000000000000001122
downloading 1_1_1_1_0000000000000000111b
downloading 1_1_1_1_0000000000000000111c
downloading 1_1_1_1_0000000000000000111d
downloading 1_1_1_1_00000000000000000c07
downloading 1_1_1_1_00000000000000000c0d
downloading 1_1_1_1_00000000000000000c0e
downloading 1_1_1_1_00000000000000000c03
downloading 1_1_1_1_00000000000000000c08
downloading 1_1_1_1_00000000000000000c09
downloading 1_1_1_1_00000000000000000c0a
downloading 1_1_1_1_00000000000000000c04
downloading 1_1_1_1_00000000000000000c05
downloading 1_1_1_1_00000000000000000c06
downloading 1_1_1_1_00000000000000000cc9
downloading 1_1_1_1_00000000000000000cca
downloading 1_1_1_1_00000000000000000ccb
downloading 1_1_1_1_00000000000000000ccc
downloading 1_1_1_1_00000000000000000ccd
downloading 1_1_1_1_00000000000000000cce
downloading 1_1_

downloading 1_1_1_1_000000000000000011b8
downloading 1_1_1_1_000000000000000011b9
downloading 1_1_1_1_000000000000000011ba
downloading 1_1_1_1_000000000000000011bb
downloading 1_1_1_1_000000000000000011be
downloading 1_1_1_1_000000000000000011bf
downloading 1_1_1_1_00000000000000000c73
downloading 1_1_1_1_00000000000000000c74
downloading 1_1_1_1_00000000000000000c75
downloading 1_1_1_1_00000000000000000c76
downloading 1_1_1_1_00000000000000000c77
downloading 1_1_1_1_00000000000000000c78
downloading 1_1_1_1_00000000000000000c79
downloading 1_1_1_1_00000000000000000c7a
downloading 1_1_1_1_00000000000000000c7b
downloading 1_1_1_1_00000000000000000c7c
downloading 1_1_1_1_00000000000000000c7d
downloading 1_1_1_1_00000000000000000c7e
downloading 1_1_1_1_00000000000000000c8d
downloading 1_1_1_1_0000000000000000118e
downloading 1_1_1_1_0000000000000000118f
downloading 1_1_1_1_00000000000000000c89
downloading 1_1_1_1_00000000000000000611
downloading 1_1_1_1_00000000000000000612
downloading 1_1_

downloading 1_1_1_1_0000000000000000037e
downloading 1_1_1_1_00000000000000000380
downloading 1_1_1_1_00000000000000000381
downloading 1_1_1_1_00000000000000000382
downloading 1_1_1_1_00000000000000000383
downloading 1_1_1_1_00000000000000000384
downloading 1_1_1_1_00000000000000000385
downloading 1_1_1_1_00000000000000000386
downloading 1_1_1_1_00000000000000000387
downloading 1_1_1_1_00000000000000000388
downloading 1_1_1_1_00000000000000000389
downloading 1_1_1_1_0000000000000000038a
downloading 1_1_1_1_0000000000000000038b
downloading 1_1_1_1_00000000000000000abc
downloading 1_1_1_1_00000000000000000abd
downloading 1_1_1_1_00000000000000000abe
downloading 1_1_1_1_00000000000000000abf
downloading 1_1_1_1_00000000000000000ae0
downloading 1_1_1_1_00000000000000000ae1
downloading 1_1_1_1_00000000000000000ae2
downloading 1_1_1_1_00000000000000000ae3
downloading 1_1_1_1_00000000000000000ab8
downloading 1_1_1_1_00000000000000001204
downloading 1_1_1_1_00000000000000001205
downloading 1_1_

EEException: Too many tasks already in the queue (3000). Please wait for some of them to complete.

In [53]:
image = ee.Image(clist.get(25))
iid = image.id().getInfo()
name = '0.25 image '+iid
print('downloading '+name)
xx = ee.batch.Export.image.toDrive(image, description = 'Beaver_Dam_Export_0.25', folder = 'Beaver Dam Images', fileNamePrefix = name, scale = 0.25).start()

downloading 0.25 image 1_1_1_1_0000000000000000004f


In [54]:
print(size)

832


In [20]:
trueColor = ee.Image(clist.get(10))
trueColorVis = {'min': 0.0, 'max': 255.0}


the_map = geemap.Map()
the_map.addLayer(trueColor, trueColorVis, 'True Color')


vis.folium_display(the_map)